In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv("../Capstone_Three_DataWranglingEDA/data_new.csv", encoding='latin-1')
data.head()

,Unnamed: 0,type,message,value,message_length
0,0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,1,ham,Ok lar... Joking wif u oni...,0,29
2,2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,3,ham,U dun say so early hor... U c already then say...,0,49
4,4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


In [3]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
data.head()

,type,message,value,message_length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


In [5]:
data.describe()

,value,message_length
count,5572.000000,5572.000000
mean,0.134063,80.118808
std,0.340751,59.690841
min,0.000000,2.000000
25%,0.000000,36.000000
50%,0.000000,61.000000
75%,0.000000,121.000000
max,1.000000,910.000000


In [12]:
#removes puncutation, stopwords, returns list of cleaned message

import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

def message_process(mess):

    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alvinqin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
data.head()

,type,message,value,message_length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61


In [14]:
data['clean_message'] = data.message.apply(message_process)
data.head()

,type,message,value,message_length,clean_message
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,Go jurong point crazy Available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry wkly comp win FA Cup final tkts 21s...
3,ham,U dun say so early hor... U c already then say...,0,49,dun say early hor c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah think goes usf lives around though


In [15]:
from collections import Counter

words = data[data.type=='ham'].clean_message.apply(lambda x: [word.lower() for word in x.split()])
ham = Counter()

for msg in words:
    ham.update(msg)
    
print(ham.most_common(50))

[('get', 303), ('ltgt', 276), ('ok', 272), ('go', 247), ('ill', 236), ('know', 232), ('got', 231), ('like', 229), ('call', 229), ('come', 224), ('good', 222), ('time', 189), ('day', 187), ('love', 185), ('going', 167), ('want', 163), ('one', 162), ('home', 160), ('lor', 160), ('need', 156), ('sorry', 153), ('still', 146), ('see', 137), ('n', 134), ('later', 134), ('da', 131), ('r', 131), ('back', 129), ('think', 128), ('well', 126), ('today', 125), ('send', 123), ('tell', 121), ('cant', 118), ('ã\x8c', 117), ('hi', 117), ('take', 112), ('much', 112), ('oh', 111), ('night', 107), ('hey', 106), ('happy', 105), ('great', 100), ('way', 100), ('hope', 99), ('pls', 98), ('work', 96), ('wat', 95), ('thats', 94), ('dear', 94)]


In [16]:
from collections import Counter

words = data[data.type=='spam'].clean_message.apply(lambda x: [word.lower() for word in x.split()])
spam = Counter()

for msg in words:
    spam.update(msg)
    
print(spam.most_common(50))

[('call', 347), ('free', 216), ('txt', 150), ('mobile', 123), ('text', 120), ('claim', 113), ('stop', 113), ('reply', 101), ('prize', 92), ('get', 83), ('new', 69), ('send', 67), ('nokia', 65), ('urgent', 63), ('cash', 62), ('win', 60), ('contact', 56), ('service', 55), ('please', 52), ('guaranteed', 50), ('customer', 49), ('16', 49), ('week', 49), ('tone', 48), ('per', 46), ('phone', 45), ('18', 43), ('chat', 42), ('awarded', 38), ('draw', 38), ('latest', 36), ('ã¥â£1000', 35), ('line', 35), ('150ppm', 34), ('mins', 34), ('receive', 33), ('camera', 33), ('1', 33), ('every', 33), ('message', 32), ('holiday', 32), ('landline', 32), ('shows', 31), ('ã¥â£2000', 31), ('go', 31), ('box', 30), ('number', 30), ('apply', 29), ('code', 29), ('live', 29)]


In [17]:
# Use countvectorizer to convert text to matrix of token counts

X = data.clean_message
y = data.value 

In [20]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer()

In [25]:
#create document term matrix

X_train_dtm = vect.transform(X_train)

In [26]:
X_train_dtm = vect.fit_transform(X_train)

In [27]:
X_train_dtm

<4179x7992 sparse matrix of type '<class 'numpy.int64'>'
	with 34772 stored elements in Compressed Sparse Row format>

In [31]:
#transform test data into document term matrix

X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7992 sparse matrix of type '<class 'numpy.int64'>'
	with 9959 stored elements in Compressed Sparse Row format>

In [44]:
#use TfidfTransformer to transform data

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)

<4179x7992 sparse matrix of type '<class 'numpy.float64'>'
	with 34772 stored elements in Compressed Sparse Row format>

In [45]:
# use Multinomial Naive Bayes model

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [46]:
nb.fit(X_train_dtm, y_train)

MultinomialNB()

In [47]:
y_pred_class = nb.predict(X_test_dtm)

In [48]:
#calculate accuracy of class predictions

from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9827709978463748

In [56]:
#print confusion matrix
#so here we can see, 1213 ham were predicted properly, and 158 spam were predicted properly.
#However 22 spams were predicted as ham

metrics.confusion_matrix(y_test, y_pred_class)

array([[1213,    0],
       [  22,  158]])

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('cv', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])
pipe.fit(X_train, y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('tfid', TfidfTransformer()),
                ('model', MultinomialNB())])

In [50]:
y_pred = pipe.predict(X_test)

In [51]:
metrics.accuracy_score(y_test, y_pred)

0.9669777458722182

In [57]:
metrics.confusion_matrix(y_test, y_pred)

array([[1213,    0],
       [  46,  134]])

In [52]:
#compare with logistic regression model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear')

In [53]:
logreg.fit(X_train_dtm, y_train)

LogisticRegression(solver='liblinear')

In [54]:
# make class predictions for X_test_dtm

y_pred_class = logreg.predict(X_test_dtm)

In [55]:
metrics.accuracy_score(y_test, y_pred_class)

0.9842067480258435

In [58]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[1213,    0],
       [  22,  158]])

In [ ]:
#After running the pipeline, we see that the model can predict with 96.69 accruacy which messages are ham or spam.
#Comparing this with the logistic model, we see that there is a 98.42 accuracy which is greater than our model.